In [2]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *

Neuron Class functionality testing

In [3]:
inputTest = np.array([1.0,1.0])

In [6]:
#zero initialization
t0NeuronZero = Neuron(2,identity)
print(t0NeuronZero.weight_vector.shape)
t0NeuronZero.forward(inputTest)

(1, 3)


0.0

In [5]:
#random initialization
t0NeuronRandom = Neuron(2,identity,method_ini="Random")
print(t0NeuronRandom.weight_vector.shape)
t0NeuronRandom.forward(inputTest)

(1, 3)


2.8594772310144005

In [ ]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
print(vector)
print(vector.shape)
t1Neuron = Neuron(vector,identity)
print(t1Neuron.weight_vector.shape)
t1Neuron.forward(inputTest)

[1.  2.  3.5]
(3,)
(1, 3)


6.5

In [ ]:
#initialization from row vector(2D array)
vector_row = vector[np.newaxis,:]
print(vector_row)
print(vector_row.shape)
t2Neuron = Neuron(vector_row,identity)
print(t2Neuron.weight_vector.shape)
t2Neuron.forward(inputTest)

[[1.  2.  3.5]]
(1, 3)
(1, 3)


6.5

In [ ]:
#initialization from column vector(2D array)
vector_column = vector[:,np.newaxis]
print(vector_column)
print(vector_column.shape)
t3Neuron = Neuron(vector_column,identity)
print(t3Neuron.weight_vector.shape)
t3Neuron.forward(inputTest)

[[1. ]
 [2. ]
 [3.5]]
(3, 1)
(1, 3)


6.5

In [11]:
#wrong activ function
vector = np.array([1.0,2.0,3.5])
errorNeuron = Neuron(vector,1)

NotImplementedError: 

In [12]:
#wrong input function
vector = np.array(["1.0","2.0","3.5"])
errorNeuron = Neuron(vector,identity)

NotImplementedError: 

# model evaluation

In [53]:
## Calculations of how many predictions are correct

def accuracy(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    assert y_true.shape == y_pred.shape,  " shapes of the arrays must match"
    return np.mean(y_true == y_pred)


# Function that shows where the model makes mistakes. Rows are true labels and columns are predicted labels.
def confusion_matrix(y_true, y_pred, num_classes=None):

    y_true = np.asarray(y_true, dtype=int)
    y_pred = np.asarray(y_pred, dtype=int)

    assert y_true.shape == y_pred.shape

    if num_classes is None:
        num_classes = max(y_true.max(), y_pred.max()) + 1

    cm = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

In [54]:
y_true = np.array([0, 1, 2, 1, 0])
y_pred = np.array([0, 2, 0, 1, 0])  # 0.6 accruacy is expected

acc = accuracy(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred, num_classes=3)

print("Accuracy:", acc)
print("Confusion matrix:\n", cm)

Accuracy: 0.6
Confusion matrix:
 [[2 0 0]
 [0 1 1]
 [1 0 0]]


In [55]:
# fake model to test the behaviour .Forward(x) returns random logits, predict(x) picks the largest logit as class

class FakeModel:

    def __init__(self, num_classes):
        self.num_classes = num_classes

    def forward(self, X):
        N = X.shape[0]
        return np.random.randn(N, self.num_classes)  # random scores

    def predict(self, X):
        logits = self.forward(X)
        return np.argmax(logits, axis=1)

In [56]:
#Below we check if model.forward and model.predict return correct shapes

def forward_shape(model, input_dim, num_classes):
    N = 5
    X = np.random.randn(N, input_dim)
    logits = model.forward(X)

    expected = (N, num_classes)
    assert logits.shape == expected, f"Expected {expected}, got {logits.shape}"
    print("forward shape test passed:", logits.shape)


def predict_shape(model, input_dim, num_classes):
    N = 7
    X = np.random.randn(N, input_dim)
    y_pred = model.predict(X)

    y_pred = np.asarray(y_pred)
    expected = (N,)

    assert y_pred.shape == expected, f"Expected {expected}, got {y_pred.shape}"
    assert np.all((y_pred >= 0) & (y_pred < num_classes))

    print("predict shape and range test passed.")

In [57]:
# adjust according to dataset
num_classes = 10
input_dim = 784       # 784(28x28) for Fashion-MNIST. While 3072 for cifar-10

fake_model = FakeModel(num_classes=num_classes)

print("Running basic model tests with Fakemodel:")
forward_shape(fake_model, input_dim, num_classes)
predict_shape(fake_model, input_dim, num_classes)

Running basic model tests with Fakemodel:
forward shape test passed: (5, 10)
predict shape and range test passed.


In [58]:
## Here we evaluate any model that has predixt(X).
def evaluate_model_on_test_set(model, X_test, y_test, num_classes):
    y_pred = model.predict(X_test)

    acc = accuracy(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred, num_classes=num_classes)

    print("Accuracy:", acc)
    print("Confusion matrix (rows=true, cols=predictions):")
    print(cm)

    return acc, cm

In [60]:
# Fake test data 
N_test = 50
X_test = np.random.randn(N_test, input_dim)
y_test = np.random.randint(0, num_classes, size=N_test)

print("Evaluating Fakemodel on fake test set:")
acc_fake, cm_fake = evaluate_model_on_test_set(fake_model, X_test, y_test, num_classes)

Evaluating Fakemodel on fake test set:
Accuracy: 0.1
Confusion matrix (rows=true, cols=predictions):
[[1 0 0 0 0 0 0 1 0 0]
 [0 2 0 0 0 3 1 0 0 1]
 [0 1 1 0 0 1 2 1 0 1]
 [0 0 1 0 1 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 2]
 [0 0 1 0 0 0 0 2 1 0]
 [0 0 0 0 4 0 0 0 0 0]
 [1 1 2 0 0 1 2 0 1 1]
 [0 1 0 0 0 0 0 2 0 0]
 [1 1 0 2 0 1 0 1 0 0]]
